In [1]:
import json
import os
from PIL import Image
import pandas as pd

In [2]:
data_dir = 'data'

In [3]:
train_df = pd.read_csv(f"{data_dir}/train.csv", names=["image", "label"])
val_df = pd.read_csv(f"{data_dir}/val.csv", names=["image", "label"])
test_df = pd.read_csv(f"{data_dir}/test.csv", names=["image", "label"])

In [4]:
train_imgs = train_df.image.values.tolist()
val_imgs = val_df.image.values.tolist()
test_imgs = test_df.image.values.tolist()

In [5]:
def convert_yolo_coco(img_list, split):
    
    categories = [
        {"id": 1, "name": "aeroplane"},
        {"id": 2, "name": "bicycle"},
        {"id": 3, "name": "bird"},
        {"id": 4, "name": "boat"},
        {"id": 5, "name": "bottle"},
        {"id": 6, "name": "bus"},
        {"id": 7, "name": "car"},
        {"id": 8, "name": "cat"},
        {"id": 9, "name": "chair"},
        {"id": 10, "name": "cow"},
        {"id": 11, "name": "diningtable"},
        {"id": 12, "name": "dog"},
        {"id": 13, "name": "horse"},
        {"id": 14, "name": "motorbike"},
        {"id": 15, "name": "person"},
        {"id": 16, "name": "pottedplant"},
        {"id": 17, "name": "sheep"},
        {"id": 18, "name": "sofa"},
        {"id": 19, "name": "train"},
        {"id": 20, "name": "tvmonitor"},
    ]

    coco_dataset = {
        "info": {},
        "licenses": [],
        "categories": categories,
        "images": [],
        "annotations": []
    }

    for image_file in img_list:
        # Load the image and get its dimensions
        image_path = f"{data_dir}/images/{image_file}"
        image = Image.open(image_path)
        width, height = image.size
        
        # Add the image to the COCO dataset
        image_dict = {
            "id": int(image_file.split('.')[0]),
            "width": width,
            "height": height,
            "file_name": image_file
        }
        coco_dataset["images"].append(image_dict)

        # Load the bounding box annotations for the image
        with open(f"{data_dir}/labels/{image_file.split('.')[0]}.txt") as f:
            annotations = f.readlines()
        
        # Loop through the annotations and add them to the COCO dataset
        for ann in annotations:
            label, x, y, w, h = map(float, ann.strip().split())
            x_min, y_min = int((x - w / 2) * width), int((y - h / 2) * height)
            x_max, y_max = int((x + w / 2) * width), int((y + h / 2) * height)
            ann_dict = {
                "id": len(coco_dataset["annotations"]),
                "image_id": int(image_file.split('.')[0]),
                "category_id": int(label) + 1, # yolo labels started from 0
                "bbox": [x_min, y_min, x_max - x_min, y_max - y_min],
                "area": (x_max - x_min) * (y_max - y_min),
                "iscrowd": 0
            }
            coco_dataset["annotations"].append(ann_dict)

    with open(f"{data_dir}/{split}_annotations.json", 'w') as f:
        json.dump(coco_dataset, f)

    print(f"{split} annotations converted from YOLO to COCO format")

        

In [6]:
convert_yolo_coco(train_imgs, split="train")

train annotations converted from YOLO to COCO format


In [7]:
convert_yolo_coco(val_imgs, split="val")

val annotations converted from YOLO to COCO format


In [8]:
convert_yolo_coco(test_imgs, split="test")

test annotations converted from YOLO to COCO format
